<a href="https://colab.research.google.com/github/RoryAyres/fpl-live-draft-board/blob/main/draft_board_script_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#v1.0

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [2]:
sheet_url = "XXXXX" # Replace with the URL to your Google Sheet
league_id = XXXXX  # Replace with your actual league ID

tab_name = str(league_id)
choices_url = f"https://draft.premierleague.com/api/draft/{league_id}/choices"
game_data_url = "https://draft.premierleague.com/api/bootstrap-static"
refresh_interval = 5  # Seconds between refreshes

import gspread
from google.auth import default
credentials, _ = default()

In [3]:
import gspread

def get_or_create_worksheet(sheet_url, tab_name):
    client = gspread.authorize(credentials)
    spreadsheet = client.open_by_url(sheet_url)
    try:
        worksheet = spreadsheet.worksheet(tab_name)
    except gspread.exceptions.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=tab_name, rows="100", cols="26") # Can replace these with exact sizes if required
    return worksheet

In [4]:
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
from gspread.utils import rowcol_to_a1

def run_draft_sync():
    position_map = {1: "Goalkeepers", 2: "Defenders", 3: "Midfielders", 4: "Forwards"}
    draft_complete = False
    worksheet = get_or_create_worksheet(sheet_url, tab_name)

    while True:
        try:
            game_data = requests.get(game_data_url).json()
            players_df = pd.DataFrame(game_data["elements"])[["id", "web_name", "element_type"]]

            choices_response = requests.get(choices_url)
            choices_data = choices_response.json()

            now_bst = datetime.utcnow() + timedelta(hours=1)
            timestamp = now_bst.strftime("%Y-%m-%d %H:%M:%S")
            worksheet.update(range_name="J1", values=[[f"⏱️ Last Updated: {timestamp}"]])

            if isinstance(choices_data, dict) and "detail" in choices_data and choices_data["detail"] == "No League matches the given query.":
                worksheet.update(range_name="B1", values=[["❌ Invalid League ID."]])
                print("Invalid league ID.")
                time.sleep(refresh_interval)
                continue

            choices = choices_data.get("choices", [])
            if len(choices) == 0:
                worksheet.update(range_name="B1", values=[["🟡 Draft Has Not Started."]])
                print("Waiting for draft to start...")
                time.sleep(refresh_interval)
                continue

            choices_df = pd.DataFrame(choices)[["entry_name", "player_first_name", "player_last_name", "element", "index", "choice_time"]]

            # If entry_name and index are present, populate headers
            if not choices_df["entry_name"].isna().all() and not choices_df["index"].isna().all():
                choices_df["player_display"] = choices_df["player_first_name"] + " " + choices_df["player_last_name"].str[0]
                first_picks = choices_df.groupby("entry_name")["index"].min().sort_values()
                manager_order = first_picks.index.tolist()
                manager_first_names = (
                    choices_df.drop_duplicates("entry_name")
                    .set_index("entry_name")["player_display"]
                    .reindex(manager_order)
                    .fillna("").to_dict()
                )

                worksheet.update(range_name="A2", values=[[""] + [manager_first_names[m] for m in manager_order]])
                worksheet.update(range_name="A3", values=[[""] + manager_order])

                # Get pick progress
                picks_made = choices_df["element"].notna().sum()
                total_picks = len(choices_df)
                worksheet.update(range_name="B1", values=[[f"🚧 Draft In Progress ({picks_made} / {total_picks} Picks)"]])
            else:
                worksheet.update(range_name="B1", values=[["🟡 Draft Has Not Started"]])
                print("Waiting for draft to start...")
                time.sleep(refresh_interval)
                continue

            # If no picks made yet, wait until at least one exists to build the board
            if choices_df["element"].notna().sum() == 0:
                print("Waiting for first pick...")
                time.sleep(refresh_interval)
                continue

            # Merge pick data and build board
            choices_df = choices_df[choices_df["element"].notna()]
            choices_df["player_display"] = choices_df["player_first_name"] + " " + choices_df["player_last_name"].str[0]
            df = choices_df.merge(players_df, left_on="element", right_on="id", how="left")
            df["position"] = df["element_type"].map(position_map)
            df["player_name"] = df["web_name"]

            # Redetermine manager order
            first_picks = df.groupby("entry_name")["index"].min().sort_values()
            manager_order = first_picks.index.tolist()
            manager_first_names = (
                choices_df.drop_duplicates("entry_name")
                .set_index("entry_name")["player_display"]
                .reindex(manager_order)
                .fillna("").to_dict()
            )

            worksheet.update(range_name="A2", values=[[""] + [manager_first_names[m] for m in manager_order]])
            worksheet.update(range_name="A3", values=[[""] + manager_order])

            df = df.sort_values(["element_type", "index"])
            output_rows = []
            for pos in [1, 2, 3, 4]:
                output_rows.append([position_map[pos]] + ["" for _ in manager_order])
                grouped = df[df["element_type"] == pos].groupby("entry_name")["player_name"].apply(list).to_dict()
                max_len = max(len(grouped.get(m, [])) for m in manager_order)
                for i in range(max_len):
                    row = []
                    for m in manager_order:
                        picks = grouped.get(m, [])
                        row.append(picks[i] if i < len(picks) else "")
                    output_rows.append([""] + row)
                output_rows.append(["" for _ in manager_order])

            worksheet.update(range_name="A5", values=output_rows)

            # Final check for completion
            if not draft_complete and picks_made == total_picks and choices_df["element"].isna().sum() == 0:
                draft_complete = True
                original_choices_df = pd.DataFrame(choices)[["choice_time"]]
                original_choices_df["choice_time"] = pd.to_datetime(original_choices_df["choice_time"])
                draft_start = original_choices_df["choice_time"].min()
                draft_end = original_choices_df["choice_time"].max()
                duration = draft_end - draft_start
                h, rem = divmod(duration.total_seconds(), 3600)
                m, s = divmod(rem, 60)
                duration_str = f"{int(h)}h {int(m)}m {int(s)}s"
                draft_complete_time = (draft_end + timedelta(hours=1)).strftime("%Y-%m-%d %H:%M:%S")
                worksheet.update(range_name="B1", values=[[f"✅ Draft Completed At {draft_complete_time} (Duration: {duration_str})"]])
                print(f"🎉 Draft completed at {draft_complete_time} (Duration: {duration_str})")
                break

            print(f"✅ Updated at {timestamp} | Picks: {picks_made}")
            time.sleep(refresh_interval)

        except Exception as e:
            print(f"❌ Error: {e}")
            time.sleep(refresh_interval)

In [ ]:
run_draft_sync()